In [2]:
"""
Genetic algorithm to pilot a car on a circuit made of several check points:
- you can rotate the car between -20 and 20 degrees
- you can thrust between 0 and 100 (percentage)
- you cannot break for now

Interesting resources:
* http://files.magusgeek.com/csb/csb_en.html
* https://www.codingame.com/blog/genetic-algorithms-coders-strike-back-game/?utm_source=codingame&utm_medium=details-page&utm_campaign=cg-blog&utm_content=csb
* https://www.codingame.com/blog/evolutionary-trajectory-optimization/?utm_source=codingame&utm_medium=details-page&utm_campaign=cg-blog&utm_content=csb
* https://www.codingame.com/multiplayer/bot-programming/coders-strike-back
* https://en.wikipedia.org/wiki/Multi-agent_system

TODO - try other approaches like:
- neural nets (regression problem): but where do you get the data? CHECK https://www.docdroid.net/NgvoYFR/csb-nn-slides.pdf
- RL (and there you might be able to use neural nets)
"""

import abc
from collections import *
import copy
from dataclasses import *
import math
import numpy as np
from typing import *

%matplotlib inline
import imageio
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy
from PIL import Image

In [89]:
Vector = np.ndarray

Angle = float
Distance = float
Duration = float
Mass = float
Speed = float

AngularSpeed = float
Thrust = float


@dataclass(frozen=True)
class Action:
    turn: AngularSpeed
    thrust: Thrust


@dataclass(frozen=True)
class Vehicle:
    mass: Mass
    position: Vector
    speed: Speed
    direction: Angle

        
@dataclass(frozen=True)
class CheckPoint:
    position: Vector
    radius: Distance
        

def get_angle(vector):
    # Get angle from a vector (x, y)
    x, y = vector
    if x > 0: return np.arctan(y / x)
    if x < 0: return math.pi - np.arctan(- y / x)
    return math.pi / 2 if y >= 0 else -math.pi / 2


def norm2(vector):
    return np.dot(vector, vector)


def norm(vector):
    return math.sqrt(norm2(vector))


def distance2(x, y):
    return norm2(x - y)


def distance(x, y):
    return norm(x - y)


def constrain_between(value, low, high):
    if value < low:
        return low
    if value > high:
        return high
    return value

        
def move(vehicle: Vehicle, action: Action, time_step: Duration) -> Vehicle:
    """
    The formula is deduced from F = m * a = thrust
    => dv / dt = thrust / m
    => v(t+dt) = v(t) + thrust / m * dt
    """        
    turn = constrain_between(action.turn, - math.pi / 8, math.pi / 8)
    thrust = constrain_between(action.thrust, 0., 1.)
    new_direction = (vehicle.direction + turn * time_step) % (2 * math.pi)
    
    # Compute the speed
    speed = np.array([
        vehicle.speed * math.cos(vehicle.direction),
        vehicle.speed * math.sin(vehicle.direction)
    ])
    
    # Compute the acceleration (thrust)
    dv_dt = np.array([
        thrust * math.cos(new_direction) / vehicle.mass,
        thrust * math.sin(new_direction) / vehicle.mass
    ])
    
    # Add the friction force (opposite to speed)
    dv_dt += np.array([
        - 0.25 * vehicle.mass * math.cos(vehicle.direction),
        - 0.25 * vehicle.mass * math.sin(vehicle.direction)
    ])
    
    # Update speed and position (include a friction force of 15%)
    new_speed = speed + dv_dt * time_step
    new_pos = vehicle.position + new_speed * time_step
    speed_norm = math.sqrt(np.dot(new_speed, new_speed))
    return Vehicle(mass=vehicle.mass, position=new_pos, speed=speed_norm, direction=new_direction)  


@dataclass(frozen=True)
class SimulationResult:
    duration: Duration
    positions: List[Vehicle]

        
@dataclass(frozen=True)
class TrackState:
    vehicle: Vehicle
    checkpoints: List[CheckPoint]
    time_step: Duration
        

class Track:
    # TODO - get_state could take an ID for the vehicle => several vehicle to compare the trajectories
    
    def __init__(self,
                 checkpoints: List[CheckPoint],
                 vehicle: Vehicle,
                 time_step: Duration):
        self.checkpoints = list(checkpoints)
        self.next_checkpoint_index = 0
        self.vehicle = vehicle
        self.time_step = time_step
    
    def is_over(self) -> bool:
        return self.next_checkpoint_index >= len(self.checkpoints)
    
    def get_state(self) -> TrackState:
        return TrackState(vehicle=self.vehicle,
                          checkpoints=self.checkpoints[self.next_checkpoint_index:],
                          time_step=self.time_step)
    
    def move(self, action: Action) -> bool:
        self.vehicle = move(self.vehicle, action, self.time_step)
        checkpoint = self.checkpoints[self.next_checkpoint_index]
        if distance(self.vehicle.position, checkpoint.position) < checkpoint.radius:
            self.next_checkpoint_index += 1
        return self.is_over()
    
    def simulate(self, agent, max_duration: Duration) -> List[Vehicle]:
        duration = Duration()
        positions = []
        while duration < max_duration and not self.is_over():
            positions.append(self.vehicle)
            state = self.get_state()
            action = agent(state)
            self.move(action)
            duration += self.time_step
        return SimulationResult(duration=duration, positions=positions)

        
class Agent(abc.ABC):
    @abc.abstractmethod
    def __call__(self, state: TrackState) -> Action:
        pass

    
class RenderedSimulation:
    # TODO - make it more beautiful
    
    def __init__(self, width: int, height: int, scale: float):
        self.width = width
        self.height = height
        self.scale = scale

    def simulate(self, track: Track, agent: Agent, max_duration: Duration, output_file_path: str):
        images = []
        image = np.zeros((self.width, self.height, 3), 'uint8')
        for checkpoint_nb, checkpoint in enumerate(track.get_state().checkpoints):
            self._draw_checkpoint(image, checkpoint, checkpoint_nb)
        simulation_result = track.simulate(agent, max_duration)
        for vehicle in simulation_result.positions:
            self._draw_vehicle(image, vehicle)
            frame = np.transpose(image, (1, 0, 2)) # height and width are inverted in an image
            images.append(Image.fromarray(frame, 'RGB'))
        imageio.mimsave(output_file_path, images)
        return simulation_result.duration
    
    def _draw_checkpoint(self, image: np.ndarray, checkpoint: CheckPoint, checkpoint_nb: int):
        lo_x, hi_y = self._to_image_coord(checkpoint.position - checkpoint.radius)
        hi_x, lo_y = self._to_image_coord(checkpoint.position + checkpoint.radius)
        center_x, center_y = self._to_image_coord(checkpoint.position)
        center = np.array([center_x, center_y])
        for x in range(lo_x, hi_x+1):
            for y in range(lo_y, hi_y+1):
                if distance(np.array([x, y]), center) < self.scale * checkpoint.radius:
                    image[x, y, min(checkpoint_nb, 1)] = 255
    
    def _draw_vehicle(self, image: np.ndarray, vehicle: Vehicle):
        center_x, center_y = self._to_image_coord(vehicle.position)
        for dx in range(-2, 2+1):
            for dy in range(-2, 2+1):
                x = center_x + dx
                y = center_y + dy
                if self._in_image(x, y):
                    image[x, y, 2] = 255
    
    def _to_image_coord(self, vector):
        x = int(vector[0] * self.scale + self.width // 2)
        y = int(vector[1] * self.scale + self.height // 2)
        return x, self.height - y - 1
    
    def _in_image(self, x, y):
        return 0 <= x < self.width and 0 <= y < self.height


# TODO - represent the genome as sequence of N pairs (angle diff, thrust), with N being the prediction depth
# TODO - investigate on how to do the mutations and the crossover
# TODO - investigate on how to transform the genome into actual moves (you have difference between genes and the behavior)

In [90]:
class CautiousAgent(Agent):
    """
    Agent that turn the wheels toward the target and accelarate (with a threshold of speed)
    """
    def __call__(self, state: TrackState) -> Action:
        vehicle = state.vehicle
        checkpoint = state.checkpoints[0]
        target_angle = get_angle(checkpoint.position - vehicle.position)
        turn = (target_angle - vehicle.direction) % (2 * math.pi)
        turn = turn if turn <= math.pi else turn - 2 * math.pi # to get angle between pi and -pi
        thrust = 1. if vehicle.speed < 2 else 0.
        return Action(turn=turn, thrust=thrust)
    

def run_first_track(agent: Agent, max_duration: Duration, output_file_path: str):
    vehicle = Vehicle(mass=1., position=np.array([0., 0.]), speed=0., direction=0.)
    checkpoints=[
        CheckPoint(position=np.array([10., -5.]), radius=1.),
        CheckPoint(position=np.array([0., -10.]), radius=1.),
        CheckPoint(position=np.array([-10., 5.]), radius=1.),
        CheckPoint(position=np.array([-2., 15.]), radius=1.)
    ]

    track = Track(checkpoints=checkpoints, vehicle=vehicle, time_step=0.2)
    simulation = RenderedSimulation(width=500, height=400, scale=6.)
    duration = simulation.simulate(track, agent, max_duration=max_duration, output_file_path=output_file_path)
    print(duration, "seconds")


run_first_track(agent=CautiousAgent(), max_duration=40., output_file_path='simulation_cautious.gif')

32.799999999999926 seconds


In [110]:
class GreedyAgent:
    # TODO - to be generalized into minimax with alpha-beta (but no opponent, does not make sense)
    
    def __init__(self):
        self.actions = [
            Action(thrust=thrust, turn=turn)
            for thrust in [0., 1.]
            for turn in [-math.pi, 0, math.pi]
        ]

    def __call__(self, state: TrackState) -> Action:
        vehicle = state.vehicle
        checkpoints = state.checkpoints
        distances_to_end = self._distances_to_end(checkpoints)
        time_step = state.time_step
        best_distance = float('inf')
        best_action = None
        for action in self.actions:
            new_vehicle = move(vehicle, action, time_step)
            new_distance = self._explore(new_vehicle, checkpoints, distances_to_end, time_step*2, 3)
            if new_distance < best_distance:
                best_distance = new_distance
                best_action = action
        return best_action
    
    def _distances_to_end(self, checkpoints: List[CheckPoint]) -> List[float]:
        n = len(checkpoints)
        distances = [0] * n
        for i in reversed(range(n - 1)):
            distances[i] = distances[i+1] + distance2(checkpoints[i].position, checkpoints[i+1].position)
        return distances
    
    def _explore(self, vehicle: Vehicle,
                 checkpoints: List[CheckPoint],
                 distances_to_end: List[float],
                 time_step: Duration,
                 depth: int):        
        best_fitness = float('inf')
        to_visit = [(vehicle, 0, time_step, depth)]
        while to_visit:
            vehicle, checkpoint_i, time_step, depth = to_visit.pop()
            checkpoint = checkpoints[checkpoint_i]
            if distance2(vehicle.position, checkpoint.position) <= checkpoint.radius ** 2:
                checkpoint_i += 1
                if checkpoint_i == len(checkpoints):
                    return 0.
                checkpoint = checkpoints[checkpoint_i]
            
            fitness = self._evaluate(vehicle, checkpoint) + distances_to_end[checkpoint_i]
            if not depth:
                best_fitness = min(best_fitness, fitness)
            else:
                for action in self.actions:
                    new_vehicle = move(vehicle, action, time_step)
                    to_visit.append((new_vehicle, checkpoint_i, time_step*2, depth-1))
        return best_fitness

    def _evaluate(self, vehicle: Vehicle, checkpoint: CheckPoint) -> float:
        return distance2(vehicle.position, checkpoint.position) + 0.2 * vehicle.speed ** 2


run_first_track(agent=GreedyAgent(), max_duration=40., output_file_path='simulation_greedy.gif')

25.79999999999994 seconds
